In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('/kaggle/input/modu-ds-4-house-prices/train.csv')
test = pd.read_csv('/kaggle/input/modu-ds-4-house-prices/test.csv')

# EDA

## Basic Structure

In [ ]:
for column in train.columns:
    if column not in test.columns:
        print(column)

* Target column: SalePrice

In [ ]:
print('Train Shape:', train.shape)
print('Test  Shape:', test.shape)

In [ ]:
house_df = train.copy()
house_df.head(3)

In [ ]:
print('\nFeatures type\n',house_df.dtypes.value_counts())

In [ ]:
import missingno as msno
msno.matrix(df = house_df)

In [ ]:
isnull_series = house_df.isnull().sum()
print('\nNull Count:\n',isnull_series[isnull_series > 0].sort_values(ascending=False))

In [ ]:
# Null 값이 포함된 열 선택
train_null_columns = house_df.columns[house_df.isnull().any(axis=0)]
print("Null 값이 포함된 칼럼:", train_null_columns.size)
print(train_null_columns)

# Null 값이 포함되지 않은 열 선택
train_not_null_columns = house_df.columns[house_df.notnull().all(axis=0)]
print("\nNull 값이 없는 칼럼:", train_not_null_columns.size)
print(train_not_null_columns)

In [ ]:
# Null 값이 포함된 열 선택
test_null_columns = test.columns[test.isnull().any(axis=0)]
print("Null 값이 포함된 칼럼:", test_null_columns.size)
print(test_null_columns)

# Null 값이 포함되지 않은 열 선택
test_not_null_columns = test.columns[test.notnull().all(axis=0)]
print("\nNull 값이 없는 칼럼:", test_not_null_columns.size)
print(test_not_null_columns)

In [ ]:
for column in train_not_null_columns:
    if column not in test_not_null_columns:
        print(column)

In [ ]:
for column in train_null_columns:
    if column not in test_null_columns:
        print(column)
        print("Is it not Null in test: ",column in test_not_null_columns)

In [ ]:
house_df[['MasVnrArea','Electrical']].isna().mean()

## Null Anaysis

* Test 데이터 셋과 Train 데이터 셋에서 Not-null 값이 포함된 칼럼은 Target Column을 제외하고 동일하다.
* Null 값을 포함하고 있는 칼럼은 train set에서 총 19개로, 그 중 두 개의 칼럼은 Test 데이터에서는 Not Null이다.
* Train Data 기준으로 Not Null 값을 포함하고 있는 칼럼은 총 62개이고 독립변수 61개, 종속변수는 1개이다.
* Train Data 기준으로 Not Null 값을 포함하고 있는 칼럼은 총 63개이고 독립변수 63개이다.
* Train 데이터에서 Null이나 Test 데이터에서는 Not Null인 칼럼은 결측치 처리 방안을 고려해 분석에 사용될 예정이다.

# Preprocess

## Target Colum

* Log-scaling process
* reason: make it follow normal distribution before regression

In [ ]:
plt.title('Original Sale Price Histogram')
plt.xticks(rotation=15)
sns.histplot(house_df['SalePrice'], kde=True)
plt.show()

In [ ]:
plt.title('Log Transformed Sale Price Histogram')
log_SalePrice = np.log1p(house_df['SalePrice'])
sns.histplot(log_SalePrice, kde=True)
plt.show()

In [ ]:
# SalePrice 로그 변환
original_SalePrice = house_df['SalePrice'] #기존의 가격정보 저장
house_df['SalePrice'] = np.log1p(house_df['SalePrice']) #스케일링된 값으로 갱신

## X_features

* Null Value가 많은 칼럼은 삭제(기준: Null Ratio 10% 이상)
* Null Value가 적은 범주형 칼럼은 Null여부를 새로운 칼럼으로 저장
* Electrical: Test 데이셋에서는 Not Null 이므로 최빈값으로 대체
* 숫자형 데이터 왜도가 높은 피쳐는 log 스케일을 적용
* 숫자형 데이터에 Standard Scaling 적용
* 원본 데이터를 99% 이상 설명할 수 있는 주성분 사용(Features -2)
* 범주형 데이터이나 실제로 숫자형을 담고 있는 경우 숫자형으로 변환
* 범주형 데이터는 One-hot Encoding 적용(Null Value는 별도의 범주로 지정)

## Null Value 처리

In [ ]:
top_freqs = []

for col in house_df.columns:
    top_freq = house_df[col].value_counts(normalize=True, dropna=False).values[0]
    top_freqs.append((col, top_freq))

# 정리해서 보기 좋게 출력
top_freq_df = pd.DataFrame(top_freqs, columns=['Columxn', 'TopFreqRatio'])
top_freq_df = top_freq_df.sort_values(by='TopFreqRatio', ascending=False)

# 90% 이상인 피처만 필터링
# print(top_freq_df[top_freq_df['TopFreqRatio'] > 0.9])

TopFrepCols = top_freq_df[top_freq_df['TopFreqRatio'] > 0.9]['Columxn'].tolist()
TopFrepCols

In [ ]:
null_ratio = house_df[train_null_columns].isna().mean()
drop_columns = null_ratio[null_ratio > 0.1].index.tolist() #Null 값의 비율이 10% 초과
drop_columns = drop_columns + TopFrepCols
drop_columns.append('Id')

print(drop_columns)

In [ ]:
house_df.drop(drop_columns, axis = 1 , inplace = True)
house_df.head()

In [ ]:
len(list(set(drop_columns)))

In [ ]:
house_df.shape

* Columns Reduction: 81 - 27 = 54

In [ ]:
msno.matrix(df = house_df)

In [ ]:
# 종속변수 정의
y_target = house_df['SalePrice']

# Null 값을 가진 컬럼만 추출
null_cols = house_df.columns[house_df.isnull().any()]

# 시각화: 3xN 형태로 subplot 그리기
n_cols = 3
n_rows = (len(null_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(6 * n_cols, 5 * n_rows))

for i, col in enumerate(null_cols):
    # Null 여부를 기준으로 Boxplot 시각화
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(x=house_df[col].isnull(), y=y_target)
    plt.title(f"Null 여부에 따른 {col} vs SalePrice")
    plt.xlabel(f"{col} is Null")
    plt.ylabel("SalePrice")

plt.tight_layout()
plt.show()

### 나머지 Null Value 처리
* 숫자형 데이터: 중간값으로 대체
* 범주형 데이터: 최빈값으로 대체

In [ ]:
house_df[null_cols].nunique()

In [ ]:
house_df[null_cols].dtypes.value_counts()

In [ ]:
# Null 값을 가지는 컬럼에 대해 Null 여부 피처 생성
for col in house_df.columns:
    if house_df[col].isnull().any():
        house_df[f'{col}_isnull'] = house_df[col].isnull().astype(int).astype('category')
        if house_df[col].dtype in ['int64', 'float64']: # 숫자형이면 중간값으로 대체
            house_df[col].fillna(house_df[col].median(), inplace=True)
        else: # 범주형이면 최빈값으로 대체
            house_df[col].fillna(house_df[col].mode()[0], inplace=True)

In [ ]:
house_df.shape

In [ ]:
msno.matrix(df = house_df)

* 결측치 전체 처리 완료

## 숫자형 칼럼 전처리

In [ ]:
# # 숫자형 컬럼만 추출
# num_cols = house_df.select_dtypes(include=['number'])
# df = house_df[num_cols]

# # 서브플롯 설정
# fig, axes = plt.subplots(nrows = 6, ncols = 6, figsize=(20, 20))
# axes = axes.flatten()  # 2D 배열을 1D로

# # 컬럼별로 히스토그램 그리기
# for i, col in enumerate(num_cols):
#     if i < len(axes):
#         sns.histplot(data = df, x = col, ax = axes[i], kde = True, bins = 50)
#         axes[i].set_title(col)
#     else:
#         # 만약 36개보다 숫자형 컬럼이 더 많다면, 이후 plot은 무시
#         break

# # 남은 빈 subplot은 꺼버리기
# for j in range(len(num_cols), len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()
# plt.savefig("Features_distrubution.png", dpi=300)

### 숫자형 데이터
* 정말로 연속적인 분포를 갖는 변수도 포함돼 있음.
* 대부분의 피쳐는 정규분포를 따르지 않는다.
* 일부 피쳐의 형식은 숫자형이지만 범주형에 대한되는 것들도 보인다.(기준: 데이터 값의 유형 15개 미만)
* 연속적, 이산적인 값을 가지고 있지만, 데이터가 왼쪽으로 치우진, Right-squid 형태의 피쳐도 존재한다.

In [ ]:
# # 숫자형 독립변수 선택 
# num_features = house_df.select_dtypes(include=['number']).columns

# # 종속변수 (y)
# y = house_df['SalePrice']

# # 서브플롯 설정
# fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(20, 20))
# axes = axes.flatten()

# # 산점도 그리기
# for i, col in enumerate(num_features):
#     if i < len(axes):
#         sns.scatterplot(x=house_df[col], y=y, ax=axes[i], alpha=0.5)
#         axes[i].set_title(f'{col} vs SalePrice', fontsize=10)
#         axes[i].set_xlabel(col)
#         axes[i].set_ylabel('SalePrice')
#     else:
#         break

# # 남은 subplot 제거
# for j in range(len(num_features), len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()
# plt.savefig("X,y.png", dpi=300)

In [ ]:
# 1. 숫자형 컬럼 전체 추출
num_cols = house_df.select_dtypes(include=['int64', 'float64']).columns

# 2. 이산형 변수 필터링 (int형 + 유일값이 적은 컬럼)
discrete_cols = [col for col in num_cols 
                 if pd.api.types.is_integer_dtype(house_df[col]) and 
                    house_df[col].nunique() <= 15]

print("이산형 변수:", discrete_cols)

In [ ]:
len(discrete_cols)

## 이산형 변수 전처리
* 숫자형이나 범주형에 해당되는 칼럼들: 범주형 칼럼으로 변환

### PoolArea
* 숫자형 데이터 > 범주형 데이터
* 처리 원인: Pool이 없는 집이 대부분이지만, 있는 경우 그에 따른 가격변환 관측됨

In [ ]:
house_df['PoolArea'].unique()

In [ ]:
house_df['PoolArea'] = (house_df['PoolArea'] > 0).astype(int).astype('category')

In [ ]:
house_df['PoolArea'].unique()

In [ ]:
discrete_cols.remove("PoolArea")
discrete_cols #숫자형 변수이지만 실질적으로는 범주형 변수

In [ ]:
house_df[discrete_cols] = house_df[discrete_cols].astype('category') #숫자형 > 범주형 변환

In [ ]:
house_df.dtypes.apply(lambda x: x.name).value_counts()

## 연속형 칼럼 전처리

* 0이 대부분인 칼럼: 0은 하나의 그룹, 0이 아닌 값은 연속형 값으로 따로 분류(기준: 0값의 비율 > 5%)
* 0이 아닌 값만 로그 변환

In [ ]:
# 연속형 변수 = 숫자형 중 float형
continuous_cols = [col for col in num_cols 
                   if (pd.api.types.is_float_dtype(house_df[col]) or 
                       house_df[col].nunique() > 15)]

print("연속형 변수:", continuous_cols)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 숫자형 변수 24개 추출 (예: house_df에서)
# numeric_cols 는 24개의 숫자형 변수 리스트라고 가정
numeric_cols = house_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 종속변수 (예: y_target) 지정
# 예시에서는 house_df['SalePrice'] 를 종속변수라고 가정
y = house_df['SalePrice']

# 서브플롯 설정
fig, axes = plt.subplots(nrows=6, ncols=4, figsize=(16, 24))
axes = axes.flatten()

# 각 숫자형 변수에 대해 Scatter Plot 그리기
for i, col in enumerate(numeric_cols[:24]):
    sns.scatterplot(x=house_df[col], y=y, ax=axes[i])
    axes[i].set_title(f'{col} vs SalePrice', fontsize=10)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('SalePrice')

# 빈 subplot 제거 (혹시 숫자형 변수가 24개보다 적을 경우 대비)
for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
zero_ratio_cols = []

for col in house_df.columns:
    if pd.api.types.is_numeric_dtype(house_df[col]):  # 숫자형만
        zero_ratio = (house_df[col] == 0).mean()
        if zero_ratio >= 0.05: #기준 비율: 5%
            zero_ratio_cols.append((col, zero_ratio))

# 결과 보기
for col, ratio in zero_ratio_cols:
    print(f"{col}: {ratio:.2%}")

In [ ]:
zero_ratio_cols = [col for col, _ in zero_ratio_cols]
house_df[zero_ratio_cols].describe()

In [ ]:
for col in zero_ratio_cols: # 0의 비율이 5%이상 칼럼에 로그 스케일 적용
    house_df[col] = house_df[col].apply(lambda x: np.log1p(x) if x > 0 else 0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 숫자형 변수 24개 추출 (예: house_df에서)
# numeric_cols 는 24개의 숫자형 변수 리스트라고 가정
numeric_cols = house_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# 종속변수 (예: y_target) 지정
# 예시에서는 house_df['SalePrice'] 를 종속변수라고 가정
y = house_df['SalePrice']

# 서브플롯 설정
fig, axes = plt.subplots(nrows=6, ncols=4, figsize=(16, 24))
axes = axes.flatten()

# 각 숫자형 변수에 대해 Scatter Plot 그리기
for i, col in enumerate(numeric_cols[:24]):
    sns.scatterplot(x=house_df[col], y=y, ax=axes[i])
    axes[i].set_title(f'{col} vs SalePrice', fontsize=10)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('SalePrice')

# 빈 subplot 제거 (혹시 숫자형 변수가 24개보다 적을 경우 대비)
for j in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

* 로그 스케일링 적용 후 분포 자체의 변화는 없지만, x의 값의 범위는 축소된 점이 관측됨.

## 왜도 보정

* 정규분포를 따르지 않는 변수들의 왜도를 최대한 따를 수 있도록 보정
* 왜도가 높은 피쳐에 로그 스케일을 적용할 경우 약 50%의 칼럼의 왜도를 보정.
* 나머지 50%의 피쳐는 로그 스케일 후에 여전히 왜도가 높은데, 모델에 따라 별로의 처리가 요구된다.
* Rilear Regression Model에는 다소 영향을 끼치나, 회귀 트리 기반 모델에 끼치는 영향은 적다.

In [ ]:
from scipy.stats import skew

# 숫자형 데이터 중 실제로 모든 값이 숫자인 컬럼만 필터링
pure_numeric_cols = house_df[numeric_cols].select_dtypes(include=[np.number]).columns
# house_df에 칼럼 index를 [ ]로 입력하면 해당하는 칼럼 데이터 세트 반환. apply lambda로 skew( ) 호출
skew_features = house_df[pure_numeric_cols].apply(lambda x : skew(x))
# skew(왜곡) 정도가 1 이상인 칼럼만 추출.
skew_features_top = skew_features[abs(skew_features) > 1] # 왜도가 1보다 큰 경우 정규분포를 따르지 않음
print(skew_features_top.sort_values(ascending=False))

In [ ]:
np.log1p(house_df[skew_features_top.index]).apply(lambda x : skew(x)) <= 1

In [ ]:
house_df[skew_features_top.index] = np.log1p(house_df[skew_features_top.index])

## 범주형 칼럼 인코딩

In [ ]:
print('get_dummies() 수행 전 데이터 Shape:', house_df.shape)

In [ ]:
house_df.dtypes.apply(lambda x: x.name).value_counts()

In [ ]:
# 카테고리형, object형 컬럼만 따로 추출
cat_cols = house_df.select_dtypes(include=['category','object']).columns
print("Category형 변수 개수: ",len(cat_cols))
print("Category형 변수 고유값 개수")
for col in cat_cols:
    print(f"\t{col}: {house_df[col].nunique()}개")

In [ ]:
house_df_ohe = pd.get_dummies(house_df, drop_first = True)
print('get_dummies() 수행 후 데이터 Shape:', house_df_ohe.shape)

In [ ]:
# 상관관계 행렬 계산
corr_matrix = house_df_ohe.corr()

# 히트맵 시각화
plt.figure(figsize=(16, 12))  # 크기 조절
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap of Features')
plt.show()

In [ ]:
corr_matrix

In [ ]:
# 상관계수 행렬 생성
corr_matrix = house_df_ohe.corr().abs()

# 상삼각 행렬만 추출 (자기 자신과의 상관 제외 및 중복 제거)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 상관계수 기준 필터링 (예: 0.8 이상인 변수쌍)
high_corr_var = [column for column in upper.columns if any(upper[column] > 0.8)]

# 필터링된 변수로만 다시 상관계수 행렬 생성
filtered_corr = house_df_ohe[high_corr_var].corr()

# 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(filtered_corr, cmap='coolwarm')
plt.title("Highly Correlated Features (|corr| > 0.8)")
plt.show()

## 차원 축소여부 검토

* 수행은 해봤지만, 칼럼을 2개 축소하는 효과밖에 없어서 적용하지 않음.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# 정규화
scaler = StandardScaler()
df = house_df_ohe.select_dtypes(include=['int64', 'float64']) # 숫자형 데이터에만 정규화 적용
X_scaled = scaler.fit_transform(df)

# PCA
pca = PCA(n_components = 0.99)  # 99% 이상 설명하는 주성분만 사용
X_pca = pca.fit_transform(X_scaled)

print(f"선택된 주성분 개수: {X_pca.shape[1]}")

* 전체 숫자형 변수 = 22
* 99% 이상 설명력 있는 숫자형 변수 = 20
* 차원축소: -2

In [ ]:
# 설명 분산 비율 (variance ratio)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance by Number of Principal Components')
plt.grid(True)

# 특정 분기점(예: 99%) 표시
plt.axhline(y=0.99, color='r', linestyle='-')
plt.axvline(x=np.argmax(cumulative_variance >= 0.99), color='r', linestyle='--')
plt.text(np.argmax(cumulative_variance >= 0.99)+1, 0.9, '99% cut-off', color='red')

plt.show()

* 물론 PCA로 연속형 변수 2개를 제거할 수 있지만, 대부분은 범주형 데이터라서 따로 진행하지 않음

## Outliar 제거

In [ ]:
plt.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=15)
plt.xlabel('GrLivArea', fontsize=15)
plt.show()

In [ ]:
# GrLivArea와 SalePrice 모두 로그 변환되었으므로 이를 반영한 조건 생성.
cond1 = house_df_ohe['GrLivArea'] > np.log1p(4000)
cond2 = house_df_ohe['SalePrice'] < np.log1p(500000)
outlier_index = house_df_ohe[cond1 & cond2].index

print('아웃라이어 레코드 index :', outlier_index.values)
print('아웃라이어 삭제 전 house_df_ohe shape:', house_df_ohe.shape)
# DataFrame의 index를 이용하여 아웃라이어 레코드 삭제.
house_df_ohe.drop(outlier_index , axis=0, inplace=True)
print('아웃라이어 삭제 후 house_df_ohe shape:', house_df_ohe.shape)

# Function Definition

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(model):
    pred = model.predict(X_test)
    pred_exp = np.expm1(pred)
    mse = mean_squared_error(np.expm1(y_test) , pred_exp)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    print('{0} 로그 변환된 MSE: {1}'.format(model.__class__.__name__,np.round(mse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [ ]:
def get_top_bottom_coef(model):
    # coef_ 속성을 기반으로 Series 객체를 생성. index는 컬럼명.
    coef = pd.Series(model.coef_, index=X_features.columns)

    # + 상위 10개 , - 하위 10개 coefficient 추출하여 반환.
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

In [ ]:
def visualize_coefficient(models):
    # 3개 회귀 모델의 시각화를 위해 3개의 컬럼을 가지는 subplot 생성
    fig, axs = plt.subplots(figsize=(24,10),nrows=1, ncols=3)
    fig.tight_layout()
    # 입력인자로 받은 list객체인 models에서 차례로 model을 추출하여 회귀 계수 시각화.
    for i_num, model in enumerate(models):
        # 상위 10개, 하위 10개 회귀 계수를 구하고, 이를 판다스 concat으로 결합.
        coef_high, coef_low = get_top_bottom_coef(model)
        coef_concat = pd.concat( [coef_high , coef_low] )
        # 순차적으로 ax subplot에 barchar로 표현. 한 화면에 표현하기 위해 tick label 위치와 font 크기 조정.
        axs[i_num].set_title(model.__class__.__name__+' Coeffiecents', size=25)
        axs[i_num].tick_params(axis="y",direction="in", pad=-120)
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=coef_concat.values, y=coef_concat.index , ax=axs[i_num])

In [ ]:
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):
    for model in models:
        # 분할하지 않고 전체 데이터로 cross_val_score( ) 수행. 모델별 CV RMSE값과 평균 RMSE 출력
        rmse_list = np.sqrt(-cross_val_score(model, X_features, y_target,
                                             scoring="neg_mean_squared_error", cv = 5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE 값 리스트: {1}'.format( model.__class__.__name__, np.round(rmse_list, 3)))
        print('{0} CV 평균 RMSE 값: {1}'.format( model.__class__.__name__, np.round(rmse_avg, 3)))

In [ ]:
from sklearn.model_selection import GridSearchCV

def print_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params,
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_features, y_target)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값: {1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                        np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [ ]:
# 모델의 중요도 상위 20개의 피처명과 그때의 중요도값을 Series로 반환.
def get_top_features(model):
    ftr_importances_values = model.feature_importances_
    ftr_importances = pd.Series(ftr_importances_values, index=X_features.columns  )
    ftr_top50 = ftr_importances.sort_values(ascending=False)[:50]
    return ftr_top50

def visualize_ftr_importances(models):
    # 2개 회귀 모델의 시각화를 위해 2개의 컬럼을 가지는 subplot 생성
    fig, axs = plt.subplots(figsize=(24,10),nrows=1, ncols=2)
    fig.tight_layout()
    # 입력인자로 받은 list객체인 models에서 차례로 model을 추출하여 피처 중요도 시각화.
    for i_num, model in enumerate(models):
        # 중요도 상위 20개의 피처명과 그때의 중요도값 추출
        ftr_top20 = get_top_features(model)
        axs[i_num].set_title(model.__class__.__name__+' Feature Importances', size=25)
        #font 크기 조정.
        for label in (axs[i_num].get_xticklabels() + axs[i_num].get_yticklabels()):
            label.set_fontsize(22)
        sns.barplot(x=ftr_top20.values, y=ftr_top20.index , ax=axs[i_num])

In [ ]:
def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value = preds[key]
        mse = mean_squared_error(y_test , pred_value)
        rmse = np.sqrt(mse)
        print('{0} 모델의 RMSE: {1}'.format(key, rmse))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수.
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')

    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장.
        test_pred[:, folder_counter] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

# Modeling

In [ ]:
y_target = house_df_ohe['SalePrice']
X_features = house_df_ohe.drop('SalePrice',axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)

## Linear Regression

### Hyperparameter Optimization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행.
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

# Skew가 높은 피처들을 로그 변환 했으므로 다시 원-핫 인코딩 적용 및 피처/타겟 데이터 셋 생성,
house_df_ohe = pd.get_dummies(house_df)
y_target = house_df_ohe['SalePrice']
X_features = house_df_ohe.drop('SalePrice',axis = 1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)

# 피처들을 로그 변환 후 다시 최적 하이퍼 파라미터와 RMSE 출력
ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_ridge = print_best_params(ridge_reg, ridge_params)
best_lasso = print_best_params(lasso_reg, lasso_params)

### Train, predict and evaluate model

In [ ]:
# 앞의 최적화 alpha값으로 학습데이터로 학습, 테스트 데이터로 예측 및 평가 수행.
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
ridge_reg = best_ridge
ridge_reg.fit(X_train, y_train)
lasso_reg = best_lasso
lasso_reg.fit(X_train, y_train)

# 모든 모델의 RMSE 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

# 모든 모델의 회귀 계수 시각화
models = [lr_reg, ridge_reg, lasso_reg]
visualize_coefficient(models)

In [ ]:
# 앞 예제에서 학습한 lr_reg, ridge_reg, lasso_reg 모델의 CV RMSE값 출력
models = [lr_reg, ridge_reg, lasso_reg]
get_avg_rmse_cv(models)

## Tree Regressiors

###  XGBoost Regressor

In [ ]:
from xgboost import XGBRegressor

xgb_params = {'n_estimators':[1000]}
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05,
                       colsample_bytree=0.5, subsample=0.8)
best_xgb = print_best_params(xgb_reg, xgb_params)

### LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor

lgbm_params = {'n_estimators':[1000]}
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, verbose = -1,
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)
best_lgbm = print_best_params(lgbm_reg, lgbm_params)

In [ ]:
# 앞 예제에서 print_best_params( )가 반환한 GridSearchCV로 최적화된 모델의 피처 중요도 시각화
models = [best_xgb, best_lgbm]
visualize_ftr_importances(models)

## Stacking Model

In [ ]:
# 개별 모델의 학습
ridge_reg = Ridge(alpha=10)
ridge_reg.fit(X_train, y_train)
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

# 개별 모델 예측
ridge_pred = ridge_reg.predict(X_test)
lasso_pred = lasso_reg.predict(X_test)

# 개별 모델 예측값 혼합으로 최종 예측값 도출
pred = 0.4 * ridge_pred + 0.6 * lasso_pred
preds = {'최종 혼합': pred,
         'Ridge': ridge_pred,
         'Lasso': lasso_pred}
#최종 혼합 모델, 개별모델의 RMSE 값 출력
get_rmse_pred(preds)

In [ ]:
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05,
                       colsample_bytree=0.5, subsample=0.8)
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4,verbose = -1,
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)

xgb_reg.fit(X_train, y_train)
lgbm_reg.fit(X_train, y_train)
xgb_pred = xgb_reg.predict(X_test)
lgbm_pred = lgbm_reg.predict(X_test)

pred = 0.6 * xgb_pred + 0.4 * lgbm_pred
preds = {'최종 혼합': pred,
         'XGBM': xgb_pred,
         'LGBM': lgbm_pred}

get_rmse_pred(preds)

## Stacking Model

In [ ]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환.
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환.
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.
Stack_final_X_train = np.concatenate((ridge_train, lasso_train,
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test,
                                     xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용.
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test , final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RMSE 값은:', rmse)

# Submission

In [ ]:
test_ohe2

In [ ]:
## sample_data를 받아오기
sample = pd.read_csv('/kaggle/input/modu-ds-4-house-prices/sample_submission.csv')

#최종 모델 예측값
ridge_pred = ridge_reg.predict(test_ohe2)
lasso_pred = lasso_reg.predict(test_ohe2)

# 개별 모델 예측값 혼합으로 최종 예측값 도출
pred = 0.4 * ridge_pred + 0.6 * lasso_pred

# 학습시 로그변환한 target을 썻으니 결과값을 다시 복원해야합니다
pred_exp = np.expm1(pred)

# sample 파일에 예측 값을 넣어주고
sample["SalePrice"] = pred_exp

# csv로 저장하기
sample.to_csv("/kaggle/working/submission.csv", index=False)